In [3]:
import boto3
import pandas as pd
import io

# Setup S3 client
s3_client = boto3.client('s3')

# Download the file into memory
response = s3_client.get_object(Bucket='cybersecurity-data-for-ml', Key='raw-data/UNSW_NB15_training-set.csv')

# Read it into pandas
df = pd.read_csv(io.BytesIO(response['Body'].read()))

# Explore
print(df.shape)
print(df.columns)
print(df.head())
print(df['label'].value_counts())

(175341, 45)
Index(['id', 'dur', 'proto', 'service', 'state', 'spkts', 'dpkts', 'sbytes',
       'dbytes', 'rate', 'sttl', 'dttl', 'sload', 'dload', 'sloss', 'dloss',
       'sinpkt', 'dinpkt', 'sjit', 'djit', 'swin', 'stcpb', 'dtcpb', 'dwin',
       'tcprtt', 'synack', 'ackdat', 'smean', 'dmean', 'trans_depth',
       'response_body_len', 'ct_srv_src', 'ct_state_ttl', 'ct_dst_ltm',
       'ct_src_dport_ltm', 'ct_dst_sport_ltm', 'ct_dst_src_ltm',
       'is_ftp_login', 'ct_ftp_cmd', 'ct_flw_http_mthd', 'ct_src_ltm',
       'ct_srv_dst', 'is_sm_ips_ports', 'attack_cat', 'label'],
      dtype='object')
   id       dur proto service state  spkts  dpkts  sbytes  dbytes       rate  \
0   1  0.121478   tcp       -   FIN      6      4     258     172  74.087490   
1   2  0.649902   tcp       -   FIN     14     38     734   42014  78.473372   
2   3  1.623129   tcp       -   FIN      8     16     364   13186  14.170161   
3   4  1.681642   tcp     ftp   FIN     12     12     628     770  13.67

In [5]:
# --- 1. Drop irrelevant columns ---
df = df.drop(columns=['id', 'attack_cat'])

# --- 2. Feature engineering BEFORE encoding/scaling ---
df['byte_ratio'] = df['sbytes'] / (df['dbytes'] + 1)
df['is_common_port'] = df['ct_dst_sport_ltm'].isin([80, 443, 22]).astype(int)
df['flow_intensity'] = (df['spkts'] + df['dpkts']) / (df['dur'] + 1e-6)

# --- 3. One-hot encode categorical columns ---
categorical_cols = ['proto', 'service', 'state']
df = pd.get_dummies(df, columns=categorical_cols)

# Convert booleans to ints
df = df.astype({col: 'int' for col in df.columns if df[col].dtype == 'bool'})

# --- 4. Scale numerical features (except label) ---
from sklearn.preprocessing import StandardScaler

numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
numerical_cols.remove('label')

scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

# --- Checks ---
print(df.shape)                                  # Final number of rows & columns
print(df.head())                                 # Preview first few rows
print(df.describe().T[['mean', 'std']])          # Confirm scaling stats
print(df[numerical_cols].mean().round(3))        # Should be ~0
print(df[numerical_cols].std().round(3))         # Should be ~1

(175341, 198)
        dur     spkts     dpkts    sbytes    dbytes      rate      sttl  \
0 -0.191029 -0.104456 -0.135769 -0.049134 -0.102726 -0.576371  0.703839   
1 -0.109485 -0.046014  0.172599 -0.046410  0.188544 -0.576345 -1.141901   
2  0.040699 -0.089845 -0.026933 -0.048527 -0.012133 -0.576734 -1.141901   
3  0.049729 -0.060624 -0.063212 -0.047016 -0.098563 -0.576737 -1.141901   
4 -0.140417 -0.075235 -0.117630 -0.047554 -0.102057 -0.576617  0.723268   

       dttl     sload     dload  ...  service_ssl  state_CON  state_ECO  \
0  1.578100 -0.389897 -0.273700  ...    -0.017874  -0.284764  -0.008273   
1  1.560002 -0.389928 -0.069233  ...    -0.017874  -0.284764  -0.008273   
2  1.560002 -0.389964 -0.252044  ...    -0.017874  -0.284764  -0.008273   
3  1.560002 -0.389958 -0.275821  ...    -0.017874  -0.284764  -0.008273   
4  1.560002 -0.389927 -0.275561  ...    -0.017874  -0.284764  -0.008273   

   state_FIN  state_INT  state_PAR  state_REQ  state_RST  state_URN  state_no  
0   

In [7]:
import sagemaker
from sagemaker import get_execution_role

# Create SageMaker session and define bucket
session = sagemaker.Session()
bucket = 'cybersecurity-data-for-ml'  # Replace with your actual S3 bucket name
processed_prefix = 'processed-data'      # Folder in S3 to store processed files

# Save preprocessed data locally
df.to_csv('preprocessed_data.csv', index=False)

# Upload to S3 inside the 'processed-data/' folder
s3_path = session.upload_data(
    path='preprocessed_data.csv',
    bucket=bucket,
    key_prefix=processed_prefix
)

print(f"Preprocessed data uploaded to: {s3_path}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Preprocessed data uploaded to: s3://cybersecurity-data-for-ml/processed-data/preprocessed_data.csv


In [8]:
import pandas as pd
import boto3
import sagemaker

# Set up session and bucket
session = sagemaker.Session()
bucket = 'cybersecurity-data-for-ml'
processed_prefix = 'processed-data'

# Download preprocessed data from S3
s3 = boto3.client('s3')
file_name = 'preprocessed_data.csv'
s3.download_file(bucket, f'{processed_prefix}/{file_name}', file_name)

# Load into pandas
df = pd.read_csv(file_name)
df.head()

,dur,spkts,dpkts,sbytes,dbytes,rate,sttl,dttl,sload,dload,...,service_ssl,state_CON,state_ECO,state_FIN,state_INT,state_PAR,state_REQ,state_RST,state_URN,state_no
0,-0.191029,-0.104456,-0.135769,-0.049134,-0.102726,-0.576371,0.703839,1.578100,-0.389897,-0.273700,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
1,-0.109485,-0.046014,0.172599,-0.046410,0.188544,-0.576345,-1.141901,1.560002,-0.389928,-0.069233,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
2,0.040699,-0.089845,-0.026933,-0.048527,-0.012133,-0.576734,-1.141901,1.560002,-0.389964,-0.252044,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
3,0.049729,-0.060624,-0.063212,-0.047016,-0.098563,-0.576737,-1.141901,1.560002,-0.389958,-0.275821,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388
4,-0.140417,-0.075235,-0.117630,-0.047554,-0.102057,-0.576617,0.723268,1.560002,-0.389927,-0.275561,...,-0.017874,-0.284764,-0.008273,1.119382,-0.940239,-0.002388,-0.10717,-0.021762,-0.002388,-0.002388


In [9]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import dump_svmlight_file
import pandas as pd

# Load data
df = pd.read_csv('preprocessed_data.csv')
X = df.drop(columns=['label'])
y = df['label']

# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# CSV for inspection
train_df = pd.concat([y_train, X_train], axis=1)
test_df = pd.concat([y_test, X_test], axis=1)
train_df.to_csv('train.csv', index=False)
test_df.to_csv('test.csv', index=False)

# LIBSVM for SageMaker - fixed version
dump_svmlight_file(X_train, y_train.values.ravel(), 'train.libsvm')
dump_svmlight_file(X_test, y_test.values.ravel(), 'test.libsvm')

In [11]:
import sagemaker

session = sagemaker.Session()
bucket = 'cybersecurity-data-for-ml'
train_prefix = 'xgboost-data/train'
test_prefix = 'xgboost-data/test'

train_input = session.upload_data('train.libsvm', bucket=bucket, key_prefix=train_prefix)
test_input = session.upload_data('test.libsvm', bucket=bucket, key_prefix=test_prefix)

print(f"Training data: {train_input}")
print(f"Testing data: {test_input}")

Training data: s3://cybersecurity-data-for-ml/xgboost-data/train/train.libsvm
Testing data: s3://cybersecurity-data-for-ml/xgboost-data/test/test.libsvm


In [14]:
from sagemaker import image_uris
from sagemaker.estimator import Estimator

xgboost_image_uri = image_uris.retrieve("xgboost", region=session.boto_region_name, version="1.3-1")

xgb = Estimator(
    image_uri=xgboost_image_uri,
    role=sagemaker.get_execution_role(),
    instance_count=1,
    instance_type='ml.m5.large',
    output_path=f's3://cybersecurity-data-for-ml/xgboost-model-output',
    sagemaker_session=session
)

xgb.set_hyperparameters(
    objective='binary:logistic',
    num_round=100,
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=1
)

In [15]:
# Train using data channels
xgb.fit({'train': train_input, 'validation': test_input})

INFO:sagemaker:Creating training-job with name: sagemaker-xgboost-2026-01-01-01-13-54-159


2026-01-01 01:13:55 Starting - Starting the training job...
2026-01-01 01:14:09 Starting - Preparing the instances for training...
2026-01-01 01:14:32 Downloading - Downloading input data......
2026-01-01 01:15:42 Downloading - Downloading the training image...
2026-01-01 01:16:08 Training - Training image download completed. Training in progress..[2026-01-01 01:16:12.624 ip-10-0-140-97.ec2.internal:7 INFO utils.py:28] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2026-01-01 01:16:12.652 ip-10-0-140-97.ec2.internal:7 INFO profiler_config_parser.py:111] User has disabled profiler.
[2026-01-01:01:16:12:INFO] Imported framework sagemaker_xgboost_container.training
[2026-01-01:01:16:12:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2026-01-01:01:16:12:INFO] No GPUs detected (normal if no gpus installed)
[2026-01-01:01:16:12:INFO] Running XGBoost Sagemaker in algorithm mode
[2026-01-01:01:16:12:INFO] files path: /opt/ml/input/data/train
[202

In [18]:
!pip install -U "xgboost==1.7.6"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.3/200.3 MB 54.9 MB/s  0:00:03m0:00:0100:01


In [20]:
import pandas as pd
import xgboost as xgb
from sklearn.metrics import accuracy_score, classification_report

# Load and convert data
train_data = pd.read_csv('train.csv', header=None, dtype=str)
test_data = pd.read_csv('test.csv', header=None, dtype=str)

# Convert all columns to numeric
train_data = train_data.apply(pd.to_numeric, errors='coerce')
test_data = test_data.apply(pd.to_numeric, errors='coerce')

# Drop any rows with NaNs
train_data = train_data.dropna()
test_data = test_data.dropna()

# Split into features (X) and labels (y)
X_train = train_data.iloc[:, 1:]
y_train = train_data.iloc[:, 0]
X_test = test_data.iloc[:, 1:]
y_test = test_data.iloc[:, 0]

# Convert to DMatrix format
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

# Set parameters and train the model
params = {
    "objective": "binary:logistic",
    "max_depth": 5,
    "eta": 0.2,
    "gamma": 4,
    "min_child_weight": 6,
    "subsample": 0.8,
    "verbosity": 1
}

model = xgb.train(params=params, dtrain=dtrain, num_boost_round=100)

# Predict
y_pred_prob = model.predict(dtest)
y_pred = [1 if p > 0.5 else 0 for p in y_pred_prob]

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9560865721862614
Classification Report:
               precision    recall  f1-score   support

         0.0       0.95      0.91      0.93     11169
         1.0       0.96      0.98      0.97     23900

    accuracy                           0.96     35069
   macro avg       0.95      0.94      0.95     35069
weighted avg       0.96      0.96      0.96     35069

